<h1> Lecture 11  

Data Science 8, Spring 2021 </h1>

<h3>
<b>
    <ul>
        <li>Prediction  </li>
        <li>Groups      </li>
        <li>Lists       </li>
        <li>Pivots      </li>
    </ul>
</b>
</h3>

In [ ]:
from datascience import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
plots.rcParams["patch.force_edgecolor"] = True

#The following allows porting images into a Markdown window
from IPython.display import Image

## Prediction ##

In [ ]:
heights = Table.read_table('heights.csv')
heights

<h3>
    <ul>
    <b><li> <tt> 'family'</tt>: Index Number of the Family (categorical)</li></b>
    <b><li> <tt> 'father'</tt>: Father's Height (inches) </li></b> 
    <b><li> <tt> 'mother'</tt>: Father's Height  (inches) </li></b> 
    <b><li> <tt> 'midparentHeight'</tt>: Weighted Average of the Parents' Heights (inches) </li></b> 
    <b><li> <tt> 'children'</tt>: Number of Children in the Family  </li></b> 
    <b><li> <tt> 'childNum'</tt>: Birth Order of the Child (e.g., 2 means second child)  </li></b> 
    <b><li> <tt> 'childHeight'</tt>: Height of the Child (inches) in Adulthood </li></b>    
    </ul> 
</h3>    

<h2> Let's explore relationship between <tt>midparentHeight</tt> and <tt>childHeight</tt>. </h2>

<h3> What's a good type of plot for this purpose? </h3>

In [ ]:
heights.scatter('midparentHeight', 'childHeight')

<h3>
Suppose we know the <tt>midparentHeight</tt> is 68 inches for a particular family.  
    
What can we say about the child's height when they turn into an adult?
</h3>

<h3> Let's look at other families whose <tt>midparentHeight</tt> are near 68 inches. </h3>

In [ ]:
heights.scatter('midparentHeight', 'childHeight')
#The following lines plot vertical reference lines
#lw denotes line width
plots.plot([67.5, 67.5], [50, 85], color='purple', lw=2)  #lw denotes line width
plots.plot([68.5, 68.5], [50, 85], color='orange', lw=2);

<h3> Now let's calculate the Average Height of the people within the region bounded by the vertical lines. </h3>

<h3> Grab only the rows where midparentHeight is in the interval   
$$67.5 \leq \texttt{midparentHeight} < 68.5.$$

</h3>

In [ ]:
# Grab only the rows where midparentHeight is in the interval 67.5 <= midparentHeight < 68.5
nearby = heights.where('midparentHeight', are.between(67.5, 68.5))  

<h3> Now take average height of the children in such families </h3>

<h2> New Function Alert: <tt>mean</tt></h2>

In [ ]:
#Take the average 
nearby_mean = nearby.column('childHeight').mean()
nearby_mean

<h3> Now insert this average in the table above.</h3>

In [ ]:
heights.scatter('midparentHeight', 'childHeight')
plots.plot([67.5, 67.5], [50, 85], color='purple', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='orange', lw=2)
#We insert a dot of size s=70, of our desired color, as a single-point superimposed on the original plot.
plots.scatter(68, nearby_mean, color='red', s=70);  # s=50 specifies the size of the dot

<h3> Now let's generalize ...</h3>

In [ ]:
def predict(midparentHeight):
    nearby = heights.where('midparentHeight', are.between(midparentHeight - 1/2, midparentHeight + 1/2))
    return nearby.column('childHeight').mean()

In [ ]:
predict(68)

In [ ]:
predict(70)

In [ ]:
predict(73)

<h3> Now <tt>apply</tt> our <tt>predict</tt> function to the entire table.</h3>

In [ ]:
predicted_heights = heights.apply(predict, 'midparentHeight')
predicted_heights

<h3> Now add the predicted values as a new column to our table.</h3>

In [ ]:
heights = heights.with_column('predictedHeight', predicted_heights)
heights

<h3>Q. Do you notice any patterns the <tt>predictedHeight</tt> column above?

<ul> 
    <b><li> Siblings get the same predicted value. </li></b>    
    <b><li> Our function tends to overestimate female heights and underestimate male heights. </li></b>    
</ul>

</h3>



In [ ]:
heights.select(
    'midparentHeight', 'childHeight', 'predictedHeight').scatter('midparentHeight')

<h3> When we call <tt>scatter</tt> with only one argument, it plots all other columns against that column specified as an argument to <tt>scatter</tt>.
    
So, here we plot <tt>childHeight</tt> and <tt>predictedHeight</tt> (yellow) against <tt>midparentHeight</tt> (horizontal axis).
    
</h3>

<h2> Prediction Accuracy: How good are our predictions? </h2>

<h3> Write a function that computes the difference between two values.</h3>

In [ ]:
def difference(x, y):
    return x - y

<h3> Now <tt>apply</tt> to the <tt>predictedHeight</tt> and <tt>childHeight</tt> columns.</h3>

In [ ]:
pred_errors = heights.apply(difference, 'predictedHeight', 'childHeight')
pred_errors

<h3> Add the errors array as a new column to our table.</h3>

In [ ]:
heights = heights.with_column('errors',pred_errors)
heights

<h3> Now create a histogram of the errors.</h3>

In [ ]:
heights.hist('errors')

<h3> Make a histogram based on gender.</h3>

In [ ]:
heights.hist('errors', group='gender')

# Discussion Question

In [ ]:
def predict_smarter(h, g):
    nearby = heights.where('midparentHeight', are.between(h - 1/2, h + 1/2))
    nearby_same_gender = nearby.where('gender', g)
    return nearby_same_gender.column('childHeight').mean()

In [ ]:
predict_smarter(68, 'female')

In [ ]:
predict_smarter(68, 'male')

In [ ]:
smarter_predicted_heights = heights.apply(predict_smarter, 'midparentHeight', 'gender')
heights = heights.with_column('smartPredictedHeight', smarter_predicted_heights)
heights

In [ ]:
smarter_pred_errs = heights.apply(difference, 'childHeight', 'smartPredictedHeight')
heights = heights.with_column('smartErrors', smarter_pred_errs)
heights

In [ ]:
heights.hist('smartErrors', group='gender')

<h1>Group</h1>

<h2> The <span style='font-family:sans-serif'> <font color='blue'> group </font> </span> method aggregates&#8212;with respect to a particular column (attribute)&#8212;all rows having the same categorical value.
<ul>
    <li> <i>First argument:</i>         Which column to group by</li>
    <li> <i>Second argument (Optional):</i> What do we want <span style='font-family:sans-serif'> <font color='blue'> group </font> </span> to do with each unique value of the attribute with respect to which the grouping takes place? </li>
    <ul>
        <li><span style='font-family:sans-serif'> <font color='blue'> len </font> </span> (default)&#8212;number of grouped values. Counts how many times each category appears.</li>
        <li> <span style='font-family:sans-serif'> <font color='blue'> list</font> </span>&#8212;list of all grouped values.</li>
        <li><span style='font-family:sans-serif'> <font color='blue'> sum</font> </span>&#8212;total of all grouped values.</li>
    </ul>
<ul>    
<h2>

<h2> Grouping by One Column </h2>

In [ ]:
cones = Table.read_table('cones.csv')
cones

<h3>Group by Flavor (we've done this sort of thing before)</h3>  

<h4> Notice, there's no second argument given to <tt>group</tt></h4>

In [ ]:
cones.group('Flavor')

<h4> Now let's take an average of the values for each column, relative to grouping based on Flavor</h4>

In [ ]:
cones.group('Flavor', np.average)

<h4>The 'Price average' column consists of entries that correspond to the average price of bubblegum, chocolate, and strawberry icecreams, respectively.</h4>  

<h4>Python doesn't know how to take an average of a categorical variable such as 'Color'.  So, let's remove the 'Color' column before we take the average.</h4>

In [ ]:
cones.drop('Color').group('Flavor', np.average)

<h4>As a second argument, we can also use <tt>min</tt>, to determine the minimum price of each ice cream flavor.  Again, since Python doesn't know how to take the minimum of the entries in the 'Color' column, we first drop that column. </h4>

In [ ]:
cones.drop('Color').group('Flavor', min)

<h3> Grouping by One Column: Welcome Survey </h3>

In [ ]:
survey = Table.read_table('welcome_survey_v1.csv')
survey

<h3> Q. How do the other variables relate to Extroversion? </h3>  

<h5>Before we group, we remove all the columns that have categorical (non-numerical) values.</h5>

In [ ]:
by_extroversion = survey.drop('Year','Handedness','Pant leg','Sleep position').group('Extroversion', np.average)
by_extroversion

<h3>We can plot the last two columns against Extroversion.</h3>

In [ ]:
by_extroversion.plot('Extroversion') # Drop the categorical columns

<h3>It's hard to tell from the plot above whether there's a trend in the Hours of sleep average.  
    
So, let's plot only that against Extroversion.</h3>

In [ ]:
by_extroversion.select(0,2).plot('Extroversion')

<h3> The above is way too dramatized.</h3>  

<h3>Rescale the vertical axis to see a more realistic variation in the number of hours of sleep. </h3>

In [ ]:
# Revisit the minimum and maximum sleep hours reported
min_sleep=min(survey.column('Hours of sleep'))
#min_sleep

#max_sleep=max(survey.column('Hours of sleep'))
#max_sleep

#by_extroversion.select(0,2).plot('Extroversion')
#plots.ylim(min_sleep, max_sleep);  # Optional for Data 8

<h1> Lists </h1>

<h3> In one sense, lists are like arrays&#8212;they're a sequence of values:
<ul> 
    <li>Separated by commas; </li>
    <li> Flanked on the two sides by a pair of square brackets [,,,]</li>
</ul>
</h3>

    
<h3>
Distinctions between <tt>array</tt> and <tt>list</tt>
<ul> 
    <li>Elements of an array <i>must be</i> of the same type.</li>
    <li> Elements of a list <i>need not be</i> of the same type.</li>
    <li> We can have nesting in a list&#8212;e.g., an array within a list, or a list within a list, etc. </li>
    <li> Arrays are natural for holding columns. </li>
    <li> Lists are natural for holding rows, since the elements across a row in a table need not be of the same type.</li>
</ul>
</h3>  

## Simple Lists

<h4>A list similar to an array</h4>

In [ ]:
[1, 5, 9]

<h4>Lists that contains multiple types of entries.</h4>

In [ ]:
[1, 5, 'hello', 5.0]

In [ ]:
[1, 5, 'hello', 5.0, make_array(1,2,3)]

<h3> Q. How many types does the following list contain? </h3>

In [ ]:
simple_list= ['hello', 7, 10.1, np.pi, Table().with_column('K', [3, 4]), True]

In [ ]:
np.pi

In [ ]:
abc=Table().with_column('K', [3, 4])
abc

In [ ]:
simple_list

<h3>Nesting a List</h3>

In [ ]:
my_array = make_array(1, 2, 3)

crowded_list = [my_array, 'what is going on', -10]
crowded_list

In [ ]:
Table().with_columns('Numbers', [1, 2, 3])

<h3> Create an empty table with only the column labels specified</h3>

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

<h3>Now construct the table row by row</h3>

<h3>New Function Alert: <tt>table_name.with_rows([])</tt> </h3>

<h4>Notice we have a list of four items, each of which is a list in its own right.</h4>

In [ ]:
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

<h1> Cross-Classification: Grouping by Two Columns </h1>

SLIDE

In [ ]:
survey = Table.read_table('welcome_survey_v1.csv')

<h2> Grouping by Two Columns </h2>

<h3>To specify the multiple columns that we care about we pass them as a list to the first argument of <tt>group</tt>.</h3>

In [ ]:
survey.group(['Handedness','Sleep position']).show()  

<h4>Q. Why do we use <tt>.show()</tt> in the cell above?</h4>

<h4>Q. Is there an association between being <i>Handedness</i> and <i>Sleep position</i>?</h4>

<h1> Pivot Tables </h1>

<h2>Pivot Tables $\longleftrightarrow$ Discern Association Between Two Categorical Variables</h2>

<h2>Scatter Plots $\longleftrightarrow$ Discern Association Between Two Numerical Variables</h2>

<h2>
Example:</h2>
    <ul>
        <li> <h3>Type of Environment A Person Lives In: Big City, Suburb, Rural.</h3></li>
        <li> <h3>Political Party Affiliation: Republican, Democratic, Libertarian, Green, Other.</h3> </li>
    </ul>
</h2>

<h4> One of these categories is assigned to the horiztonal axis of the Pivot Table, and the other is assigned to the vertical axis, thereby creating a $3\times 5$ or $5\times 3$ grid. 

SLIDE: Pivot Tables

<h3>New Function Alert: <tt>table_name.pivot(List of Columns)</tt></h3>

In [ ]:
survey.pivot('Sleep position', 'Handedness')

<h4>Each combination corresponds to a different location in the grid table above.</h4>

<h3> How to Read the Table</h3>

<h4> Suppose we want to determine if Left-Handed people are likelier to sleep on their left or right sides.</h4>  

<h4>This is how we read the table. </h4>

![title](pivot-table-min-syntax.png)

<h3> Below is an example with all four arguments of the <tt>.pivot()</tt> function.</h3>

In [ ]:
survey.pivot('Sleep position', 'Handedness', values='Extroversion', collect=np.average)

<h3>The way we read the table above</h3>

![title](pivot-table-full-syntax.png)

<h1> Aggregating Numerical Variables </h1>

<h2> Summarize a Group by More Than a Mere Head Count</h2>

In [ ]:
survey.group('Handedness', collect=np.average)

<h3> Let's remove the extraneous columns that clutter the table:</h3>

In [ ]:
survey.drop('Year','Pant leg','Sleep position').group('Handedness', collect=np.average)

In [ ]:
survey.group('Sleep position', collect=np.average)

<h3> Let's remove the extraneous columns that clutter the table:</h3>

In [ ]:
survey.drop('Year','Handedness','Pant leg').group('Sleep position', collect=np.average)

<h3> Q. Do people who sleep on their backs tend to get more sleep than those who sleep on their right side? </h3>

In [ ]:
survey.drop('Year','Handedness','Pant leg').group('Sleep position', collect=np.average).select(0,3).plot('Sleep position')
#Uncomment the line below to rescale the plot and see the macro picture
#plots.ylim(min_sleep, max_sleep);  # Optional for Data 8

<h3> Let's combine 'Handedness' and 'Sleep position' </h3>

In [ ]:
t = survey.select('Handedness', 'Sleep position', 'Hours of sleep')
t.group(['Handedness','Sleep position'], collect=np.average).show()

<h3> Q. On average, how many hours of sleep do folks who are left-handed AND sleep on their left sides get?</h3>

<h3> How does that compare to the average number of hours of sleep that folks who are left-hand AND sleep on their right sides?</h3>

<h2>Aggregation on Pivot Tables</h2>

<h3>Instead of looking at the combinations 'Sleep position' and 'Handedness' as rows (like in the table above), let's put them into a grid.</h3>

In [ ]:
survey.pivot('Sleep position', 'Handedness', values='Hours of sleep', collect=np.average)

<h3>IMPORTANT: If I specify one of the optional arguments, we must specify both.</h3>

<h3>For example, if we omit the <tt>collect</tt> argument, we get an error. </h3>

In [ ]:
# Here, pivot doesn't know how to combine all the hours of sleep
# for each subgroup of students
survey.pivot('Sleep position', 'Handedness', values='Hours of sleep')

<h1> Challenge Question </h1>

SLIDE: Challenging Question

In [ ]:
sky = Table.read_table('skyscrapers_v2.csv')
sky = (sky.with_column('age', 2020 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

In [ ]:
# 1. For each city, what’s the tallest building for each material?










In [ ]:
# 2. For each city, what’s the height difference between the tallest 
#    steel building and the tallest concrete building?












Don't read ahead until you try the challenge questions yourself first!

In [ ]:
sky.select('material', 'city', 'height').group(['city', 'material'], collect=max)

In [ ]:
sky_p = sky.pivot('material', 'city', values='height', collect=max)
sky_p.show()

In [ ]:
sky_p = sky_p.with_column(
    'difference', 
    abs(sky_p.column('steel') - sky_p.column('concrete'))
)
sky_p

In [ ]:
sky_p.sort('difference', True)

### Take-home question: try it here!

In [ ]:
# Generate a table of the names of the oldest buildings for each 
# material for each city:

